<a href="https://colab.research.google.com/github/AeroEng16/machine-learning-with-python-logistic-regression-3211129/blob/main/02_nnclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tutorial link - https://youtu.be/V_xro1bcAuA
https://learnpytorch.io

## Neural Network Classification
**Got to 8:57 in video**